## Monster Dungeon Game <br>
<p>We'll be creating a Dungeon Monster game together, that increases in difficulty as the levels persist. The object of the game is to collect all of the eggs in the level and reach the door before getting eating by the monster(s) in the level. The game should be modular so that you can easily implement a larger scale game, or make the game more difficult.</p>
<p>TODO: Save player/game information in file so that players can access saved games later.</p>

#### Monster Dungeon Class Creation

In [5]:
class MonsterDungeon():
    def __init__(self, level, player_name, game_obj, cols=2, rows=2):
        self.cols = cols
        self.rows = rows
        self.level = level
        self.player_name = player_name
        self.game_obj = game_obj
    
        self.cols += self.level
        self.rows += self.level
        
    def initialGrid(self):
        player = Player(self.player_name,self.addPos(),self.cols,self.rows)  
        self.addObj(player)
        door = Door(self.addPos())
        self.addObj(door)
        basket = Basket(self.addPos())
        self.addObj(basket)
        
        num_monsters = 2**(self.level-1)
        for i in range(num_monsters):
            monster = Monster(self.addPos(),self.cols,self.rows)
            self.addObj(monster)
        for i in range(self.level):
            egg = Egg(self.addPos())
            self.addObj(egg)
            
    def getLevel(self):
        return self.level
    
    def setLevel(self, diff):
        self.level += diff
    
    def getCoords(self):
        return [obj.getCoords() for obj in self.game_obj]
    
    def addPos(self):
        coord = [randint(0,self.cols-1),randint(0,self.rows-1)]
        while coord in self.getCoords():
            coord = [randint(0,self.cols-1),randint(0,self.rows-1)]
        return coord
        
    def addObj(self, obj):
        self.game_obj.append(obj)
    
    def makeGrid(self):
        for obj in self.game_obj:
            if obj.getType() == 'Player': 
                print(f'{obj.getType()} Lives: {obj.getLives()}\n')
            print(f'{obj.getType()} Coordinates: {obj.getCoords()}')
            
        for row in range(self.rows):
            print('+---' * self.cols + '+')
            for col in range(self.cols):
                e = ''
                if col == self.cols-1:
                    e = '|'
                    
                noObj = True
                for obj in self.game_obj:
                    if obj.getCoords() == [col, row]:
                        noObj = False
                        symbol = obj.getType()[0].lower()
                        print(f'| {symbol} ', end=e)
                        break
                if noObj:
                    print('|   ', end=e)
            print('')
        # bottom border
        print('+---' * self.cols + '+')
    
    def checkCollision(self, obj1, obj2):
        return obj1.getCoords() == obj2.getCoords()
    
    def handleMonsterHit(self, player, monster):
        if self.checkCollision(player, monster):
            player.loseALife()
            print(f'You got eaten! Lives left: {player.lives}')
    
    def getObjs(self, objName):
        return [obj for obj in self.game_obj if obj.getType() == objName]
    
    def checkBasket(self, player, basket):
        if self.checkCollision(player, basket):
            self.game_obj.pop(2)
            print('Retrieved basket!')
    
    def checkEgg(self, player):
        for egg in self.getObjs('Egg'):
            if self.checkCollision(player, egg):
                if len(self.getObjs('Basket')) > 0:
                    print('Retrieve basket first!')
                else:
                    self.game_obj.pop()
                    numEggs = len(self.getObjs('Egg'))
                    print(f'Retrieved egg! Eggs remaining: {numEggs}')
                    break

    def checkWinCondition(self, player, door):
        if self.checkCollision(player, door):
            if len(self.getObjs('Egg')) == 0:
                print('You win!')
                self.setLevel(1)
                return False
            else:
                print(f'Retrieve all eggs! Eggs remaining: {len(self.getObjs("Egg"))}')
        return True
    
    def checkLoseCondition(self, player):
        if player.getLives() <= 0:
            print(f'You lose, {player.getName()}!')
            return False
        return True

#### Class Creation for Monsters, Player, Egg & Door

In [6]:
class GameObj():
    def __init__(self,coords):
        self.coords = coords
    
    def getCoords(self):
        return self.coords

    def getType(self):
        return type(self).__name__
    
class Player(GameObj):
    moves = {'up': [1,-1], 'down': [1,1], 'left': [0,-1], 'right': [0,1]}
    def __init__(self, name, coords, cols, rows, lives=3):
        self.name = name
        self.lives = lives
        self.cols = cols
        self.rows = rows
        super().__init__(coords) # [col,row]
        
    def getName(self):
        return self.name
        
    def movePlayer(self, move):
        if move in Player.moves:
            rowcol,diff = Player.moves[move]
            self.coords[rowcol] += diff
            for i in [0,1]:
                if self.coords[i] < 0:
                    self.coords[i] = [self.cols,self.rows][rowcol] - 1
                elif self.coords[i] >= [self.cols,self.rows][rowcol]:
                    self.coords[i] = 0
        else:
            print('Invalid move.')
    
    def loseALife(self):
        self.lives -= 1
        
    def getLives(self):
        return self.lives

class Monster(GameObj):
    def __init__(self, coords, cols, rows):
        self.cols = cols
        self.rows = rows
        super().__init__(coords)
    
    def moveMonster(self):
        self.coords = [randint(0,self.cols-1),randint(0,self.rows-1)]

class Door(GameObj):
    def __init__(self, coords):
        super().__init__(coords)

class Basket(GameObj):
    def __init__(self, coords):
        super().__init__(coords)

class Egg(GameObj):
    def __init__(self, coords):
        super().__init__(coords)

#### Main Loop

In [15]:
from IPython.display import clear_output
from random import randint

# global variables
player_name = input('Your name: ').strip().title()
level = 1
max_level = 6

# main loop
while level < max_level:
    playing = True
    start_level = level
    
    game = MonsterDungeon(level, player_name, [])
    game.initialGrid()

    # game loop
    while playing:
        # show grid
        game.makeGrid()

        # ask player for movement or quit
        ans = input('What would you like to do? (up/down/left/right/quit) ').strip().lower()

        clear_output()

        if ans == 'quit':
            playing = False
            print('Thanks for playing!')
        else:
            player = game.getObjs('Player')[0]
            door = game.getObjs('Door')[0]
            player.movePlayer(ans)
            for monster in game.getObjs('Monster'):
                monster.moveMonster()
            if len(game.getObjs('Basket')) > 0:
                basket = game.getObjs('Basket')[0]
                game.checkBasket(player,basket)
            if len(game.getObjs('Egg')) > 0:
                game.checkEgg(player)
            
            playing = game.checkWinCondition(player, door)
            
            if playing:
                for monster in game.getObjs('Monster'):
                    game.handleMonsterHit(player, monster)

                playing = game.checkLoseCondition(player)
    
    # ask to play again
    level = game.getLevel()
    if level == max_level:
        print(f'Congrats {player_name}! You have cleared all the levels!')
        if input('Start over from level 1? (yes/no) ').strip().lower() == 'yes':
            game.setLevel((1-max_level))
            level = game.getLevel()
            clear_output()
        else:
            break
    elif start_level < level:
        if input('Proceed to next level? (yes/no) ').strip().lower() == 'no':
            game.setLevel(-1)
            level = game.getLevel()
            if input('Play again? (yes/no) ').strip().lower() == 'no':
                print ('Thanks for playing!')
                break
            else:
                clear_output()
        else:
            clear_output()
    else:
        if input('Play again? (yes/no) ').strip().lower() == 'no':
                print ('Thanks for playing!')
                break
        else:
            clear_output()

You got eaten! Lives left: 0
You lose, Billy Bob Thorton!
Play again? (yes/no) no
Thanks for playing!
